In [89]:
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt
from transformers import pipeline
import jaro

Notes:

Spacy for dependency parsing. (still need to do)

In [127]:
def regex_clean(text):
    clean_text = re.sub(r"^.*?:\s*|\n", "", text).strip()
    clean_text = re.sub(r"\b\d+/?\d*\s*\w+\b\s*|\bper\s+\w+\b\s*|-|,.+", "", re.sub(r"\/", " ", clean_text)).strip()
    
    return clean_text

def compute_accuracy(product_type, product_name, results_series, regex_results_series):
    """ Inputs:
            product_type: original column of input text
            product_name: original column of labels
            results_series: pandas series object after QA model applied
            regex_results_series: pandas series object after regex function applied
            
        Output:
            (QA model accuracy, Regex function accuracy)
            
        functionality: 
            computes jaro winkler score between transformed text and
            original labels, if the score is above 60% counts as a success
            then divides by the length of the orinal inputs
    """

    num_correct_qa = 0
    num_correct_reg = 0

    for i in range(len(product_type)):
        qa_pred = str(results_series[i])
        reg_pred = str(regex_results_series[i])
        label = str(product_name[i])
        qa_score = jaro.jaro_winkler_metric(qa_pred, label)
        reg_score = jaro.jaro_winkler_metric(reg_pred, label)
        
        print("QA ENTRY:{:<80} LABEL:{:<50} SCORE:{:<30}".format(qa_pred, label, qa_score))
        
        print("RE ENTRY:{:<80} LABEL:{:<50} SCORE:{:<30}".format(reg_pred, label, reg_score))
        
        if qa_score > 0.6:
            num_correct_qa += 1
        
        if reg_score > 0.6:
            num_correct_reg += 1
            
    return ("QA", num_correct_qa/len(product_type)), ("RE", num_correct_reg/len(product_type))

Upload datasets

In [128]:
product_type = pd.read_csv('CONFIDENTIAL_GFPP sample dataset for ML algorithm.xlsx - Sample dataset.csv')['Product Type']
product_type1 = pd.read_csv('CONFIDENTIAL_GFPP sample dataset for ML algorithm.xlsx - Sample dataset.csv')['Product Type']
product_name = pd.read_csv('CONFIDENTIAL_GFPP sample dataset for ML algorithm.xlsx - Sample dataset.csv')['Product Name']

In [130]:
text = product_type[104]
text

'ROAST CHICKEN DINNER 4 OZ. ROASTED CHICKEN\n(BONELESS 4 OZ.), 2 OZ. OF GRAVY, 4 OZ. CARROTS\nTZIMMES, 4 OZ. POTATO PUDDING\nC-PET OVENABLE/MICROWAVEABLE PACKAGING\n14 OZ. CONTAINERS, 12 CONTAINERS PER CASE.\nBRAND OFFERED: MEAL MART'

Instantiate Model and Parameters

In [61]:
qa_model = pipeline("question-answering")
#question = "what food does this label refer to?"
question = "What food is this talking about?"

Apply QA model and regex functions to original inputs, (product_type, product_type1)

In [84]:
results_series = product_type.apply(lambda x: qa_model(question = question, context = x)['answer'])

In [119]:
regex_results_series = product_type1.apply(lambda x: regex_clean(x))

In [126]:
compute_accuracy(product_type, product_name, results_series, regex_results_series)

QA ENTRY:General Mills Cheerios Cereal                                                    LABEL:cereal, cheerios, ss                               SCORE:0.6446360153256704            
RE ENTRY:General Mills Cheerios Cereal  bowls  Gluten Free                                LABEL:cereal, cheerios, ss                               SCORE:0.6046768707482993            
QA ENTRY:Cereal - 1oz bowls                                                               LABEL:cereal, honey scooters, ss                         SCORE:0.6174566174566175            
RE ENTRY:General Mills Honey Nut Scooters Cereal  bowls  Gluten Free                      LABEL:cereal, honey scooters, ss                         SCORE:0.5942240433765857            
QA ENTRY:bowls - Gluten Free                                                              LABEL:cereal, raisin bran, ss                            SCORE:0.5062929061784897            
RE ENTRY:General Mills Raisin Bran Cereal  1.bowls  Gluten Free                 

(('QA', 0.07081789683670561), ('RE', 0.07594756340837845))

In [ ]:
for text in product_type:
    
    clean_text = re.sub(r"\n", " ", text).strip()
    clean_text = re.sub(r"\b\d+/?\d*\s*\w+\b\s*|\bper\s+\w+\b\s*|-|,.+", "", re.sub(r"\/", " ", text)).strip()
    clean_text = qa_model(question = question, context = text)['answer']
    print(clean_text)

In [ ]:
#Writing a Text Classifier with 9 custom labels using huggingface transfomrers library.

In [131]:
"""UPLOAD DATA"""

data = pd.read_csv('CONFIDENTIAL_GFPP sample dataset for ML algorithm.xlsx - Sample dataset.csv')


In [139]:
print(data['Food Product Category'].unique())

len(data['Food Product Category'].unique())


['Grain Products' 'Condiments & Snacks' 'Tree Nuts & Seeds' 'Beverages'
 'Fruit' 'Meals' 'Vegetables' 'Seafood' 'Chicken' 'Beef'
 'Turkey, Other Poultry' 'Roots & Tubers' 'Butter' 'Cheese' 'Eggs'
 'Milk & Dairy' 'Yogurt' 'Legumes' 'Non-Food' 'Rice' 'Pork' 'Milk'
 'Fish (Wild)' 'Produce' 'Fish (Farm-Raised)' 'Meat']


26